In [1]:
# Import libraries
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import CountVectorizer, BERTTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
import re
import json
from sklearn.metrics import confusion_matrix
import pickle


# Initialization
lemmatizer = WordNetLemmatizer()
# count_vectorizer = CountVectorizer(stop_words="english", ngram_range=(1, 1))
# BERT_transformer = BERTTransformer()
nb_clf = MultinomialNB()

tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
tag_map['AS'] = wn.ADJ_SAT

# filepath = "finalized_8K_accounts.csv"
# filepath = "finalized_8K_accounts_emojis_replaced.csv"
# filepath = "FINALIZED_Training_Data_ALL_Available_Descriptions_EMOJIS_REPLACED.csv"
filepath = "FINALIZED_Training_Data_ALL_Available_Descriptions_EMOJIS_UNCHANGED.csv"

hand_label = "hand.label_simplified"

df = pd.read_csv(filepath)

# Removing all the "-int" (international, non-English, descriptions)
#dict.fromkeys(df[hand_label])
df = df[((df[hand_label] == 'media') | (df[hand_label] == 'tourbiz') |(df[hand_label] == 'acad') | (df[hand_label] == 'gov') | (
        df[hand_label] == 'other'))]

df = df[['username', 'description', hand_label]]  # keep only relevant columns

words_not_changed = ['media']

result = {}
n_gram_range = (1, 1)


def preprocessing(row):
    if str(row) == "nan":
        lemma = ""
    else:
        row = str(row).lower()
        row = word_tokenize(row)  # tokenize
        lemma = [lemmatizer.lemmatize(token, tag_map[tag[0]]) if token not in words_not_changed else token for
                 token, tag in pos_tag(row)]  # lemmatization, depending on part-of-speech
        lemma = ["" if re.search(r'\b[0-9]+\b\s*', lem) else lem for lem in lemma]  # removing
    return str(lemma)


df['description_lemmatized'] = df['description'].apply(preprocessing)

# Remove all the empty descriptions
df = df[df['description_lemmatized'] != ""]
#df[hand_label]
#print(df.shape)
#df[df['description_lemmatized'] != ""].shape

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Sentences are encoded by calling model.encode()
#print(type(df[['description']]))
embeddings = model.encode(df['description'].tolist())

In [8]:
import keras
import scikeras
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.layers import Dense, LSTM
#from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasClassifier, KerasRegressor

In [12]:
# create a function that returns a model, taking as parameters things you
# want to verify using cross-valdiation and model selection
def create_model(neurons, optimizer='rmsprop', kernel_initializer='glorot_uniform'):
    model = Sequential()
    # model.add(Dense(neurons,activation='relu',kernel_initializer=kernel_initializer, input_dim=384))
    # Add 2 bidirectional LSTMs
    model.add(LSTM(neurons, return_sequences=True, input_shape=(384, 1)))  # returns a sequence of vectors of dimension 32
    #model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
    model.add(LSTM(32))  # return a single vector of dimension 32
    model.add(Dense(units = 5, activation = 'softmax', kernel_initializer=kernel_initializer))

    model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

    return model

##Reproducibility
seed = np.random.seed(22)
rng = np.random.RandomState(1)

# wrap the model using the function you created
clf = KerasClassifier(build_fn=create_model, neurons = 10, random_state=rng, validation_split=0.2, verbose=False, callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=20)])

# just create the pipeline
pipeline = Pipeline([
    ('normalize', StandardScaler(with_mean=False)),
    ('clf',clf)
])

In [13]:
X = embeddings
y_labels = df[hand_label]
y_orig_labels = df[hand_label]

from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()
y_labels = label_as_binary.fit_transform(y_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.2, random_state=42, stratify=y_labels)

In [14]:


## CV grid search
param_BERT_grid = [
    {
        'clf__neurons': [10, 50, 100, 200],
    }
]

# !!! Does STRATIFICATION BY DEFAULT !!!
grid_search_BERT = GridSearchCV(pipeline, param_BERT_grid, cv=3, scoring='accuracy', verbose=0)
grid_search_BERT.fit(X_train, y_train)

/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/hom

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('normalize',
                                        StandardScaler(with_mean=False)),
                                       ('clf',
                                        KerasClassifier(build_fn=<function create_model at 0x7f415008d430>, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f41047d45e0>], neurons=10, random_state=RandomState(MT19937) at 0x7F4280335040, validation_split=0.2, verbose=False))]),
             param_grid=[{'clf__neurons': [10, 50, 100, 200]}],
             scoring='accuracy')

In [15]:
grid_search_BERT.best_params_

{'clf__neurons': 10}

In [16]:
pipeline.set_params(**grid_search_BERT.best_params_)
our_model = pipeline.fit(X_train, y_train)

/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


In [17]:
y_pred_cv = cross_val_predict(our_model, X_train, y_train, cv=10)

INFO:tensorflow:Assets written to: ram://2c0544277eec45febdc93cdebc378184/assets


INFO:tensorflow:Assets written to: ram://2c0544277eec45febdc93cdebc378184/assets
2024-03-19 17:29:12.742277: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://26aa8cbd2adb48e684148c1f0857a420: INVALID_ARGUMENT: ram://26aa8cbd2adb48e684148c1f0857a420 is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: ram://ab65fddb700146aca48c7c49d3e5d5d2/assets


INFO:tensorflow:Assets written to: ram://ab65fddb700146aca48c7c49d3e5d5d2/assets
2024-03-19 17:30:00.036024: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://bf307366227c4e33a4e2485eb6173cfe: INVALID_ARGUMENT: ram://bf307366227c4e33a4e2485eb6173cfe is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: ram://0fd2abbf187741918914c1b640c1bac7/assets


...

INFO:tensorflow:Assets written to: ram://fb5ca15690264f22bbf04de9fdba346e/assets


2024-03-19 17:36:37.965058: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://6737d66f029c4850bd2fbec207eace03: INVALID_ARGUMENT: ram://6737d66f029c4850bd2fbec207eace03 is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: ram://5ab03b7f0036412c9b16e00456ae826e/assets


INFO:tensorflow:Assets written to: ram://5ab03b7f0036412c9b16e00456ae826e/assets


2024-03-19 17:37:34.771320: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://02e69ece58014486ad63981b77041c86: INVALID_ARGUMENT: ram://02e69ece58014486ad63981b77041c86 is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


In [18]:
y_pred_cv_orig = label_as_binary.inverse_transform(y_pred_cv)
y_train_orig = label_as_binary.inverse_transform(y_train)

In [23]:
np.unique(y_pred_cv_orig)

array(['other'], dtype='<U7')

In [19]:
print(metrics.classification_report(y_train_orig, y_pred_cv_orig))

              precision    recall  f1-score   support

        acad       0.00      0.00      0.00       430
         gov       0.00      0.00      0.00       103
       media       0.00      0.00      0.00      1299
       other       0.79      1.00      0.88      7551
     tourbiz       0.00      0.00      0.00       155

    accuracy                           0.79      9538
   macro avg       0.16      0.20      0.18      9538
weighted avg       0.63      0.79      0.70      9538



/home/andrey/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andrey/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andrey/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
y_test_pred = our_model.predict(X_test)
y_test_pred_orig = label_as_binary.inverse_transform(y_test_pred)

y_test_orig = label_as_binary.inverse_transform(y_test)

print(metrics.classification_report(y_test_orig, y_test_pred_orig))

              precision    recall  f1-score   support

        acad       0.00      0.00      0.00       108
         gov       0.00      0.00      0.00        25
       media       0.00      0.00      0.00       325
       other       0.79      1.00      0.88      1888
     tourbiz       0.00      0.00      0.00        39

    accuracy                           0.79      2385
   macro avg       0.16      0.20      0.18      2385
weighted avg       0.63      0.79      0.70      2385



/home/andrey/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andrey/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andrey/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
